# Проект "Исследование рынка заведений общественного питания Москвы" #
Цель проекта:
- на основании исследования рынка заведений общественного питания в Москве, подготовить презентацию для привлечения инвестиций в открытие кафе в Москве

# Cодержание #

<a id='data_description'></a>
# Описание данных #
Таблица rest_data:
id — идентификатор объекта;
object_name — название объекта общественного питания;
chain — сетевой ресторан;
object_type — тип объекта общественного питания;
address — адрес;
number — количество посадочных мест.

<a id='analyse_plan'></a>
# План проведения исследования #
1. загрузка данных
2. подготовка данных
3. анализ данных

<a id='import'></a>
# Импорт библиотек #

In [12]:
import io

import pandas as pd
from IPython.display import display
import numpy as np
from scipy import stats as st
import plotly_express as px
import os
import warnings
import requests

pd.set_option('display.max_colwidth', None)
warnings.filterwarnings('ignore')

<a id='data'></a>
# Загрузка данных #

In [13]:
url="https://jupyterhub.praktikum-services.ru/user/user-0-374174871/edit/street_data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')), sep=';')
print(c)

# data_1 = pd.read_csv('https://jupyterhub.praktikum-services.ru/user/user-0-374174871/edit/street_data.csv', sep=';')
# print(data_1)
# data_street = pd.read_csv(filepath_or_buffer='datasets/street_data.csv', sep=';')
# data_street

ParserError: Error tokenizing data. C error: Expected 1 fields in line 43, saw 2


In [2]:
path_1 = 'datasets/'
path_2 = '/datasets/'
file = 'rest_data.csv'
path = ''

if os.path.exists(path_1):
    path = path_1 + file
elif os.path.exists(path_2):
    path = path_2 + file
else: print('Данные отсутствуют. Проверьте путь к папкам с данными')

data = pd.read_csv(filepath_or_buffer=path)
display(data)

,id,object_name,chain,object_type,address,number
0,151635,СМЕТАНА,нет,кафе,"город Москва, улица Егора Абакумова, дом 9",48
1,77874,Родник,нет,кафе,"город Москва, улица Талалихина, дом 2/1, корпус 1",35
2,24309,Кафе «Академия»,нет,кафе,"город Москва, Абельмановская улица, дом 6",95
3,21894,ПИЦЦЕТОРИЯ,да,кафе,"город Москва, Абрамцевская улица, дом 1",40
4,119365,Кафе «Вишневая метель»,нет,кафе,"город Москва, Абрамцевская улица, дом 9, корпус 1",50
...,...,...,...,...,...,...
15361,208537,Шоколадница,да,кафе,"город Москва, 3-й Крутицкий переулок, дом 18",50
15362,209264,Шоколадница,да,кафе,"город Москва, улица Земляной Вал, дом 33",10
15363,209186,Шоколадница,да,кафе,"город Москва, улица Земляной Вал, дом 33",20
15364,221900,Шоколадница,да,кафе,"город Москва, поселение Московский, Киевское шоссе, 22-й километр, домовладение 4, строение 1",36


In [3]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15366 entries, 0 to 15365
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           15366 non-null  int64 
 1   object_name  15366 non-null  object
 2   chain        15366 non-null  object
 3   object_type  15366 non-null  object
 4   address      15366 non-null  object
 5   number       15366 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 720.4+ KB
None


In [4]:
print(data.chain.unique())
print(data.object_type.unique())

['нет' 'да']
['кафе' 'столовая' 'закусочная' 'предприятие быстрого обслуживания'
 'ресторан' 'кафетерий' 'буфет' 'бар' 'магазин (отдел кулинарии)']


## Итоги предварительного осмотра данных ##
1. 'object_name':
   - перевести данные в нижний регистр
2. 'chain'.dtype(object) -> 'chain'.dtype(category)
3. 'object_type'.dtype(object) -> 'object_type'.dtype(category)
4. 'address':
   - перевести в нижний регистр
   - выделить отдельный столбец с названием улицы

<a id='prepare'></a>
# Предобработка данных #

<a id='prepare_change'></a>
## Изменение данных ##

In [5]:
data['object_name'] = data['object_name'].str.strip().str.lower()
data['address'] = data['address'].str.strip().str.lower()

<a id='prepare_type'></a>
## Изменение типов данных ##

In [6]:
data['chain'] = data['chain'].replace(to_replace=['да', 'нет'], value=['сетевое', 'несетевое']).astype('category')
data['object_type'] = data['object_type'].astype('category')

<a id='prepare_street'></a>
## Создание нового столбца ##

In [7]:
data['street'] = data['address'].apply(lambda x: str.split(x, ',')[1]).str.strip()

<a id='prepare_dup'></a>
## Поиск дубликатов/пропусков данных ##

In [8]:
print(data.duplicated().sum())
print(data.isna().sum())

0
id             0
object_name    0
chain          0
object_type    0
address        0
number         0
street         0
dtype: int64


## Итоги предобработки данных ##
1. переведены в нижний регистр
   - data['object_name']
   - data['address']
2. изменен тип данных:
   - data['chain']
   - data['object_type']
3. создан новый столбец:
   - data['street']

<a id='analyse'></a>
# Анализ данных #

<a id='analyse_object_kind'></a>
## Соотношение видов объектов общественного питания ##

In [9]:
fig = px.pie(data_frame=data.groupby(by='chain', axis='index')\
                            .agg({'id':'count'})\
                            .reset_index()\
                            .rename(columns={'chain':'type',
                                             'id':'count'}),
             names='type',
             values='count',
             template='seaborn',
             title='Отношение количества сетевых/несетевых заведений')
fig.update_layout(legend_title='Вид заведения')
fig.show()


## Итоги осмотра соотношения видов заведений ##
1. на рынке общественного питания Москвы преобладают несетевые заведения.
   Доли распределены следующим образом:
   - несетевые заведения - 80.7%(12398 шт.)
   - сетевые заведения - 19.3%(2968 шт.)

## Соотношение типов объектов общественного питания ##

In [10]:
fig = px.pie(data_frame=data.groupby(by='object_type', axis='index')\
                            .agg({'id':'count'})\
                            .reset_index()\
                            .rename(columns={'id':'count'}),
             names='object_type',
             values='count',
             template='seaborn',
             title='Соотношение типов заведений')
fig.update_layout(legend_title = 'Тип заведения')
fig.show()

## Итоги осмотра соотношения типов объектов общественного питания ##
1. 83.9% объектов общественного питания составляют следующие типы:
   - кафе - 39.7%(6099 шт.)
   - столовая - 16.8%(2587 шт.)
   - ресторан - 14.9%(2285 шт.)
   - предприятие быстрого обслуживания - 12.5%(1923 шт.
2. оставшуюся долю - 16.1% составляют следующие типы
   - бар
   - буфет
   - кафетерий
   - закусочная
   - магазин(отдел кулинарии)

<a id='analyse_kind_of_type'></a>
## Виды заведений характерные для типов заведений ##

In [11]:
fig = px.bar(data_frame=data.pivot_table(index=['object_type', 'chain'],
                                         values=['id'],
                                         aggfunc='count').reset_index(),
       x='object_type',
       y='id',
       color='chain',
       barmode='group',
       template='seaborn',
       text_auto=True,
       labels={'id':'Кол-во заведений',
               'object_type':'Тип заведения',
               'chain':'Вид заведения'},
       title='Виды заведений характерные для типов заведений')
fig.show()

## Итоги осмотра характерных видов заведений для типов заведений ##
1. Сетевой вид заведения наиболее характерен для следующих типов заведений:
   - кафе
   - предприятие быстрого обслуживания
   - ресторан

<a id='analyse_net_seat'></a>
## Характеристика сетевых заведений по количеству посадочных мест ##

In [12]:
centroid = data['number'].mean()
data['seat_type'] = data['number'].apply(lambda x: 'много(>=60)' if x >= centroid else 'мало(<60)')
data['seat_type'] = data['seat_type'].astype('category')

In [13]:
fig = px.pie(data_frame=data.query('chain == "сетевое"')
                            .groupby(by='seat_type')
                            .agg({'id':'count'})
                            .reset_index()
                            .rename(columns={'id':'count'}),
             names='seat_type',
             values='count',
             template='seaborn',
             title='Характеристика сетевых заведений по количеству посадочных мест')
fig.update_layout(legend_title = 'Кол-во мест')
fig.show()

## Итоги осмотра характеристики сетевых заведений по количеству посадочных мест ##
Сетевые заведения предпочитают небольшие залы, с количеством посадочных мест <60.

<a id='analyse_seat_type'></a>
## Среднее количество мест характерное для типа заведения ##

In [14]:
fig = px.bar(data_frame=data.groupby(by='object_type')
                            .agg({'number':np.mean})
                            .reset_index()
                            .sort_values(by='number', ascending=False),
             x='object_type',
             y='number',
             template='seaborn',
             text_auto='3.0f',
             color='object_type',
             title='Среднее количество мест характерное для типа заведения',
             labels={'number':'Ср. кол-во мест',
                     'object_type':'Тип заведения'})
fig.show()

## Итоги осмотра среднего количества мест характерного для типа заведения ##
1. Будем ориентироваться на заведения типа "кафе" и близкие к ним:
   - кафе ~40 посадочных мест
   - бар ~43 посадочных места
   - ресторан ~97 посадочных мест
2. Самое большое среднее количество посадочных мест предоставляет:
   - столоваяя

<a id='analyse_map'></a>
## Распределение заведений по территории Москвы ##

In [21]:

#data['address'].to_csv(path_or_buf='datasets/address.csv')

In [ ]:
# geojson =
# fig = px.choropleth_mapbox(data, geojson=geojson, color="Bergeron",
#                            locations="district", featureidkey="properties.district",
#                            center={"lat": 45.5517, "lon": -73.7073},
#                            mapbox_style="carto-positron", zoom=9)
# fig.show()

In [16]:
data

,id,object_name,chain,object_type,address,number,street,seat_type
0,151635,сметана,несетевое,кафе,"город москва, улица егора абакумова, дом 9",48,улица егора абакумова,мало(<60)
1,77874,родник,несетевое,кафе,"город москва, улица талалихина, дом 2/1, корпус 1",35,улица талалихина,мало(<60)
2,24309,кафе «академия»,несетевое,кафе,"город москва, абельмановская улица, дом 6",95,абельмановская улица,много(>=60)
3,21894,пиццетория,сетевое,кафе,"город москва, абрамцевская улица, дом 1",40,абрамцевская улица,мало(<60)
4,119365,кафе «вишневая метель»,несетевое,кафе,"город москва, абрамцевская улица, дом 9, корпус 1",50,абрамцевская улица,мало(<60)
...,...,...,...,...,...,...,...,...
15361,208537,шоколадница,сетевое,кафе,"город москва, 3-й крутицкий переулок, дом 18",50,3-й крутицкий переулок,мало(<60)
15362,209264,шоколадница,сетевое,кафе,"город москва, улица земляной вал, дом 33",10,улица земляной вал,мало(<60)
15363,209186,шоколадница,сетевое,кафе,"город москва, улица земляной вал, дом 33",20,улица земляной вал,мало(<60)
15364,221900,шоколадница,сетевое,кафе,"город москва, поселение московский, киевское шоссе, 22-й километр, домовладение 4, строение 1",36,поселение московский,мало(<60)
